In [1]:
import json
import openai
import pandas as pd
import tiktoken as tk
import tenacity
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import multiprocessing
enc = tk.encoding_for_model('gpt-3.5-turbo')

data = pd.read_json('/home/ubuntu/CodeGen/BCE/FileStructure/Python/PythonSelected/all_functions.json')
token_limit = 3500
openai.api_key = api_key

new_df = pd.DataFrame()
gpt_outputs = []


In [2]:
print("Original Input Shape: " + str(data.shape))
data.columns

Original Input Shape: (9543, 15)


Index(['filePath', 'numLines', 'lineStart', 'lineEnd', 'numParams',
       'signature', 'comment', 'numCommentLines', 'content', 'parentClass',
       'packageName', 'imports', 'repository', 'tokenCount', 'summary'],
      dtype='object')

In [6]:
import openai
import tenacity
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
gpt_outputs = []

@retry(
    stop=stop_after_attempt(15),
    wait=wait_random_exponential(multiplier=2, min=2, max=150),
)
def get_gpt_response(index):
  summary = ''
  with open(f'{index}.txt', 'r') as f:
    summary = f.read()
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "user", "content": f"""I will provide you with a prompt to a function below these instructions. You will output exactly as follows, with the list as well. Text encased in <like this> will be replaced by your response, and text encased in (like this) is just a description for the response that you do not need to type up:
  (a) <Boolean> (Is it bioinformatics related?)
  (b) <words> (Give a list of 5 keywords of why it is bioinformatics related)
  (c) <integer> (Your confidence from 0 to 100 that your response in A is accurate, so for example, if you believe strongly that it is not bioinformatics related, you should also rate a high confidence level)
  The code must explicitly reference some bioinformatics methodology, terminology, or process. For example, an AVL Tree would not be a valid bioinformatics function, while a FASTQ processor would. The keywords are defined as important words that allowed you to make the determination that the function is bioinformatics related. The confidence should be your estimate of how confident you are of your responses.

  Make sure that in your response is explicitly as follows in the directions. Part A should only be one word and a boolean, either True or False. Part B should only be 5 words, no additional information, Part C should only be a single integer, from 0 to 100, it is a measure of your confidence in your response to Part A.

  After selecting keywords, please reverify that the words you used to make the decision for Part A is actually bioinformatics related.
  Again, as clarification, I will be providing the function. 

  The responses should be formatted as a list:
  Entry 1: The response to part A converted into a string
  Entry 2: A list of 5 words which are strings from the response to Part B
  Entry 3: The integer response to part C converted to a string
  Therefore, your output should follow this guideline. This will be your only output, there should be no additional outputs beyond the one highlighted in this prompt.

  Prompt begins here:
  Here is the function:
  {data.iloc[index]['content']}
  \n Here is the summary:
  {summary}
  Prompt ends here.
  Give the output to the above code encased in "Prompt begins here:" and "Prompt ends here." Your keyword search should only encompass the words in the prompt, and ensure that keywords are related to bioinformatics, not statistics.
  """}]
  )
  return completion.choices[0].message['content']

for index, row in tqdm(data.iterrows(), total=len(data), desc="Processing rows"):
  gpt_outputs.append(get_gpt_response(index))

Processing rows:   0%|          | 0/9543 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '45616f69caf14889bd80178562de6a28b4d67c793f31663abc7c4951dafad020.txt'

In [ ]:
print(len(gpt_outputs))
gpt_outputs = [value for value in gpt_outputs if value != 'AAAAAAAAA']
print(data.shape)
print(data.columns)
print(data.tail(5))

print(data.columns)
print(len(gpt_outputs))
data.drop(data.tail(1).index, inplace=True)
data['gpt_output'] = gpt_outputs
print(data.columns)

1026
(1027, 22)
Index(['filePath', 'totalNumLines', 'lineStart', 'lineEnd', 'numParameters',
       'signature', 'comments', 'numComments', 'content', 'parentClass',
       'packageName', 'imports', 'repoAuthor', 'repoName', 'numChar',
       'addImports', 'returnType', 'license', 'numCommentLines', 'prompt',
       'good_length', 'Token Length'],
      dtype='object')
                                               filePath  totalNumLines   
1044          ./github-repos/spladder/spladder/reads.py             14  \
1045   ./github-repos/spladder/spladder/viz/coverage.py             41   
1046   ./github-repos/spladder/spladder/viz/coverage.py             64   
1047   ./github-repos/spladder/spladder/viz/coverage.py            135   
1048  ./github-repos/paleomix/paleomix/common/fileut...             12   

      lineStart  lineEnd  numParameters   
1044        635      648              3  \
1045        125      165             21   
1046        168      231             13   
1047       

In [ ]:
print(data.shape)
print(data.columns)
print(data.tail(5))


(1026, 23)
Index(['filePath', 'totalNumLines', 'lineStart', 'lineEnd', 'numParameters',
       'signature', 'comments', 'numComments', 'content', 'parentClass',
       'packageName', 'imports', 'repoAuthor', 'repoName', 'numChar',
       'addImports', 'returnType', 'license', 'numCommentLines', 'prompt',
       'good_length', 'Token Length', 'gpt_output'],
      dtype='object')
                                              filePath  totalNumLines   
1043          ./github-repos/spladder/spladder/hdf5.py             39  \
1044         ./github-repos/spladder/spladder/reads.py             14   
1045  ./github-repos/spladder/spladder/viz/coverage.py             41   
1046  ./github-repos/spladder/spladder/viz/coverage.py             64   
1047  ./github-repos/spladder/spladder/viz/coverage.py            135   

      lineStart  lineEnd  numParameters   
1043          4       42              5  \
1044        635      648              3   
1045        125      165             21   
1046    

In [ ]:
t = 0
for index, row in data.iterrows():
    gpt_output = row['gpt_output']
    if len(gpt_output) >= 3 and gpt_output[2] == 'F':
        pass
    else:
        print(str(index) + 'T')
        print(row['comments'])
        print('\n\n\n\n\n')



0T
Transform width into a half-window size.

`width` is either a fraction of the length of `x` or an integer size of the
whole window. The output half-window size is truncated to the length of `x`
if needed.






1T
Convolve a weighted window over a weighted signal array.

Source: https://stackoverflow.com/a/46232913/10049






2T
Convolve a weighted window over array `signal`.

Input array is assumed padded by `_pad_array`; output has padding removed.






3T
Choose a reasonable window size given the signal.

Inspired by Silverman's rule: bandwidth is proportional to signal's standard
deviation and the length of the signal ^ 4/5.






4T
Smooth the values in `x` with the Kaiser windowed filter.

See: https://en.wikipedia.org/wiki/Kaiser_window

Parameters
----------
x : array-like
    1-dimensional numeric data set.
width : float
    Fraction of x's total length to include in the rolling window (i.e. the
    proportional window width), or the integer size of the window.






5T
S

In [ ]:
import ast
num_edit = [105, 202, 203, 428, 466, 473, 555, 587, 588, 664, 713, 714, 715, 720, 804, 808, 824, 870, 871, 892, 1001, 1026, 1027, 1028]
print(data.loc[555]['gpt_output'])
print(data.loc[555]['comments'])
print(len(num_edit))
run_on_this = data
print(len(run_on_this))
gpt_value = run_on_this.loc[555]['gpt_output']
print(gpt_value)
gpt_value = ast.literal_eval(gpt_value)
print(gpt_value[0])
gpt_value[0] = 'True'
print(gpt_value)
gpt_value = str(gpt_value)
run_on_this.loc[555, 'gpt_output'] = gpt_value
print(run_on_this.loc[555]['gpt_output'])

for location in num_edit:
    gpt_value = run_on_this.loc[location]['gpt_output']
    gpt_value = ast.literal_eval(gpt_value)
    gpt_value[0] = 'True'
    gpt_value = str(gpt_value)
    run_on_this.loc[location, 'gpt_output'] = gpt_value
print(run_on_this['gpt_output'])
print(len(run_on_this['gpt_output']))

run_on_this.to_json('/home/ubuntu/Bio-Code-Eval/Bio-Code-Eval-new/final_python_functions_for_testing.json')

['True', ['licence', 'name', 'acronym', 'strip', 'return'], '80']
Choose the contigs to cover the reference gene.

In this stitching iteration we are assembling the exon with gaps.
24
1026
['True', ['licence', 'name', 'acronym', 'strip', 'return'], '80']
True
['True', ['licence', 'name', 'acronym', 'strip', 'return'], '80']
['True', ['licence', 'name', 'acronym', 'strip', 'return'], '80']
0       ['True', ['numpy', 'series', 'fraction', 'weig...
1       ['True', ['weighted', 'signal', 'array', 'conv...
2       ['True', ['convolve', 'weighted', 'array', 'bi...
3       ['True', ['window', 'size', 'signal', 'Silverm...
4       ['True', ['Kaiser window', 'smooth', 'numeric ...
                              ...                        
1043    ['True', ['dictionary', 'recursively', 'copy',...
1044    ['True', ['BED', 'compressed', 'contigs', 'seq...
1045    ['True', ['BED', 'BAM file', 'linearly', 'proc...
1046    ['True', ['OptionParser', 'logging', 'config',...
1047    ['True', ['list', 'l

In [ ]:
print(run_on_this.loc[555]['prompt'])

NameError: name 'run_on_this' is not defined